In [ ]:
## README!!!!! ##

## Welcome to the Avoro Mega Darknet Trainer, Making Yolo Easy!! ##

# Step by step instructions to use this program:

# 1)  Change your colab settings to use GPU by going
#     Edit > Notebook settings > Hardware accelerator > GPU

# 2)  Edit the variables in the "VARIABLES" section below the readme

# 3)  Use https://github.com/AlexeyAB/Yolo_mark to mark your images and
#     then zip just the folder containing the .jpg and .txt files which
#     should be the "x64/Release/data/img/" folder

# 4)  Upload the zip of images to github the ColabFileDownload github
#     (https://github.com/jamesabsolom/ColabFileDownload.git)

# 5)  Run the VARIABLES/SETUP section below by just clicking play
#     on the code snippet

## README!!!!! ##

In [ ]:
## VARIABLES ##
## ------------ ##

# The number of unique objects you are detecting e.g. Giraffe, Potato, Car
Classes = 1
# A list containing the name of each of the unique objects you are detecting
Classes_Names = ["Daffodil Head"]

## SETUP SCRIPT ##
## ------------ ##
import os
import shutil
import time

# Clone darknet repo
%cd /content
!git clone https://github.com/AlexeyAB/darknet

# Change makefile to have GPU and OPENCV enabled
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Verify CUDA is installed
!/usr/local/cuda/bin/nvcc --version

# Download all the files for running the training
%cd /content/darknet
!git clone https://github.com/jamesabsolom/ColabFileDownload.git

# Make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
%cd /content/darknet
!make -j8

# Move all the files from the colab file download to the correct places
classfolder = "/content/darknet/ColabFileDownload/" + str(Classes) + "class/"
os.rename(classfolder + "obj.data", "/content/darknet/build/darknet/x64/data/obj.data")
os.rename(classfolder + "yolo-obj.cfg", "/content/darknet/build/darknet/x64/cfg/yolo-obj.cfg")

# Make the names file
%cd /content/darknet/build/darknet/x64/data
with open("obj.names", 'w') as file:
    for item in Classes_Names:
        file.write(str(item) + '\n')

# Unzip the images zip
%cd /content/darknet/ColabFileDownload
os.mkdir("/content/darknet/build/darknet/x64/data/obj")
shutil.unpack_archive("/content/darknet/ColabFileDownload/img.zip", "/content/darknet/build/darknet/x64/data/obj/")

# Literally only had to add this bit cause of stupid unzip rules
%cd /content/darknet/build/darknet/x64/data
with open("train.txt", 'w') as file:
  for item in os.listdir("/content/darknet/build/darknet/x64/data/obj/img"):
    os.rename("/content/darknet/build/darknet/x64/data/obj/img/" + item, "/content/darknet/build/darknet/x64/data/obj/" + item)
    if str(item).endswith(".jpg"):
      file.write("/content/darknet/build/darknet/x64/data/obj/" + str(item) + '\n')

os.rmdir("/content/darknet/build/darknet/x64/data/obj/img")

# Download the weights file
%cd /content/darknet/build/darknet/x64
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
%cd /content/darknet
! ./darknet detector train /content/darknet/build/darknet/x64/data/obj.data /content/darknet/build/darknet/x64/cfg/yolo-obj.cfg /content/darknet/build/darknet/x64/yolov4.conv.137 -dont_show